Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [18]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [19]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [20]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Greedy + 2 opt 

In [21]:


# Initialization
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = [int(city)]

# Greedy approach for initial tour
while not np.all(visited):
    dist[:, city] = np.inf  # Mark current city's row as visited
    closest = np.argmin(dist[city])  # Find the nearest city
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city, closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))

# Close the loop back to the start
tsp.append(tsp[0])
logging.info(f"Initial path found with {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")



def two_opt_swap(route):
    best_route = route.copy()
    best_cost = tsp_cost(best_route)
    improvement = True

    while improvement:
        improvement = False
        for i in range(1, len(route) - 2):  # Avoid first and last elements
            for j in range(i + 1, len(route) - 1):
                if j - i == 1: continue  # Skip adjacent cities

                # Create a new route by swapping two edges
                new_route = route[:i] + route[i:j][::-1] + route[j:]
                new_cost = tsp_cost(new_route)
                
                # Update if we found a better route
                if new_cost < best_cost:
                    best_cost = new_cost
                    best_route = new_route
                    improvement = True
                    logging.debug(f"Improvement found with 2-opt: new length {best_cost:.2f}km")
        route = best_route

    return best_route

# Apply 2-opt to improve the initial greedy solution
optimized_tsp = two_opt_swap(tsp)
logging.info(f"Optimized path: Found a path of {len(optimized_tsp)-1} steps, total length {tsp_cost(optimized_tsp):.2f}km")


DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

In [ ]:
import itertools

def brute_force_tsp(cities):
    """Find the shortest route using brute-force method."""
    best_route = None
    min_cost = float('inf')

    # Generate all permutations of city indices, adding the start city at the end to complete the route
    for perm in itertools.permutations(range(1, len(cities))):  # Fix the starting city to reduce computation
        route = [0] + list(perm) + [0]  # Start and end at the first city
        cost = tsp_cost(route)
        
        if cost < min_cost:
            min_cost = cost
            best_route = route

    return best_route, min_cost

# Finding the best route using the brute-force method
best_route, best_cost = brute_force_tsp(CITIES)

# Logging the results
logging.info(f"Optimal path found: {best_route} with total length {best_cost:.2f} km")